# Поиск похожих изображений
## Черпал идеи:
https://habr.com/ru/post/206264/ <br>
https://habr.com/ru/post/120562/ <br>
https://medium.com/@senior_sigan/similar-images-search-ce433491059b#3344 <br>
https://youtu.be/3nct0Kalk2I?t=4108 <br>
https://ru.wikipedia.org/wiki/SSIM <br>
https://www.phash.org/ <br>
https://habr.com/ru/post/198338/ <br>
https://pillow.readthedocs.io/en/stable/reference/Image.html

In [3]:
import os
import numpy as np
from PIL import Image, ImageFilter

## pHash 
для поиска модифицированных изображений

In [81]:
def main_p(name1):
    img1 = np.array(np.asarray(Image.open(f'{name1}').resize((32,32), Image.ANTIALIAS).convert('L')),dtype=np.double)
    dct_matrix = np.zeros((32,32))
    for i in range(32):
        for j in range(32):
            ij = 0
            for n1 in range(32):
                temp = 0
                for n2 in range(32):
                    temp += img1[n1][n2]*np.cos(np.pi * (n1 + 0.5) * i / 32) * np.cos(np.pi * (n2 + 0.5) * j / 32)
                ij += temp
            dct_matrix[i][j] = ij
            
    res = dct_matrix[:8, :8]
    res_c = np.copy(res)
    res_c[0][0] = 0
    mean_res = res_c.mean()
    
    return (np.where(res < mean_res, 0, 1)).reshape((64,))

In [82]:
def hamm(a,b):
    return np.count_nonzero(a != b)

In [83]:
def pHash(path = './dev_dataset'):
    list_of_images = os.listdir(path)
    k = len(list_of_images)
    res = np.zeros((k,k))
    hash_of_images = []
    
    for i in list_of_images:
        hash_of_images.append(main_p(path+'/'+i))
    
    for i in range(k):
        for j in range(i+1,k):
            res[i][j] = hamm(hash_of_images[i], hash_of_images[j])

    return  hash_of_images

#### Определяю порог для расстояния Хэмминга
(с помощью f-score)

In [111]:
def f_mera(matrix):
    tp = matrix[0][0]
    fp = matrix[0][1]
    tn = matrix[1][1]
    fn = matrix[1][0] 
    
    prec = tp / (tp + fp)
    rec  = tp / (tp + fn)
    F = (2*prec*rec) / (prec + rec)
    
    return prec, rec, F

In [112]:
def test(path = './dev_dataset'):
    list_of_images = os.listdir(path)
    k = len(list_of_images)
    hash_of_images = []
    
    for i in list_of_images:
        hash_of_images.append(main_p(path+'/'+i))
    
    for h in range(30):
        res = np.zeros((k,k))
        for i in range(k):
            for j in range(i+1,k):
                res[i][j] = hamm(hash_of_images[i], hash_of_images[j]) < h

        matrix_of_mistakes = np.zeros((2,2))
        for i in range(k):
            for j in range(i+1,k):
                if list_of_images[i][:2] == list_of_images[j][:2]:
                    if res[i][j] == 1:
                        matrix_of_mistakes[0][0] += 1
                    else:
                        matrix_of_mistakes[1][0] += 1 
                elif res[i][j] == 1:
                    matrix_of_mistakes[0][1] += 1    
                else:
                    matrix_of_mistakes[1][1] += 1
                    
        prec, rec, F = f_mera(matrix_of_mistakes)
        print(f"{h}: F--{'%.2f'%F},prec--{'%.2f'%prec}, rec--{rec}, TP-{matrix_of_mistakes[0][0]}, FP-{matrix_of_mistakes[0][1]}, FN-{matrix_of_mistakes[1][0]}, TN-{matrix_of_mistakes[1][1]}, sum-{np.sum(res)}")

    return res
              
test5 = test('./global_test')

0: F--nan,prec--nan, rec--0.0, TP-0.0, FP-0.0, FN-32.0, TN-1096.0, sum-0.0
1: F--0.51,prec--1.00, rec--0.34375, TP-11.0, FP-0.0, FN-21.0, TN-1096.0, sum-11.0
2: F--0.61,prec--1.00, rec--0.4375, TP-14.0, FP-0.0, FN-18.0, TN-1096.0, sum-14.0
3: F--0.69,prec--1.00, rec--0.53125, TP-17.0, FP-0.0, FN-15.0, TN-1096.0, sum-17.0
4: F--0.69,prec--1.00, rec--0.53125, TP-17.0, FP-0.0, FN-15.0, TN-1096.0, sum-17.0
5: F--0.72,prec--1.00, rec--0.5625, TP-18.0, FP-0.0, FN-14.0, TN-1096.0, sum-18.0
6: F--0.72,prec--1.00, rec--0.5625, TP-18.0, FP-0.0, FN-14.0, TN-1096.0, sum-18.0
7: F--0.75,prec--1.00, rec--0.59375, TP-19.0, FP-0.0, FN-13.0, TN-1096.0, sum-19.0
8: F--0.77,prec--1.00, rec--0.625, TP-20.0, FP-0.0, FN-12.0, TN-1096.0, sum-20.0
9: F--0.77,prec--1.00, rec--0.625, TP-20.0, FP-0.0, FN-12.0, TN-1096.0, sum-20.0
10: F--0.77,prec--1.00, rec--0.625, TP-20.0, FP-0.0, FN-12.0, TN-1096.0, sum-20.0
11: F--0.79,prec--1.00, rec--0.65625, TP-21.0, FP-0.0, FN-11.0, TN-1096.0, sum-21.0
12: F--0.81,prec--1

C:\Users\milan\Anaconda3\envs\ipykernel_py3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


Видим, что очень хорошо работает алгоритм с порогом h = [17, 18, 19, 20, 21] <br>
Я дальше использую h = 21

## Хэш-функция по среднему 
###### для поиска похожих изображений (смещение точки обзора)

Ищу случаи, когда место "обзора" было немного смещенно. Для этого беру центр одного изображения и сравниваю с различными кусками другого изображения с помощью обычной хэш-функция по среднему 

In [7]:
def creator(path_of_image):
    image1 = np.array(np.asarray(Image.open(f'{path_of_image}').resize((10, 10), Image.ANTIALIAS).convert('L')),dtype=np.double)
    return np.where(image1 > image1.mean(), 1, 0)

In [8]:
def hamm(ar1, ar2):
    return np.count_nonzero(ar1.reshape((64,)) != ar2.reshape((64,)))

In [9]:
def eq(ar1, ar2, h):
    return hamm(ar1, ar2) <= h

In [10]:
def shift(ar1, ar2, h):
    
    def equality(ar1, ar2):
        return eq(ar1, ar2, h)
        
    return  equality(ar1, ar2[0:8,  0:8])  or \
            equality(ar1, ar2[0:8,  1:9])  or \
            equality(ar1, ar2[2:10, 1:9])  or \
            equality(ar1, ar2[2:10, 2:10]) or \
            equality(ar1, ar2[0:8,  2:10]) or \
            equality(ar1, ar2[2:10, 0:8])  or \
            equality(ar1, ar2[1:9,  0:8])  or \
            equality(ar1, ar2[1:9,  2:10])

In [11]:
def is_equal_with_shift(img1, img2, h):
    centr1 = img1[1:9, 1:9]
    centr2 = img2[1:9, 1:9]
    
    return shift(centr1, img2, h) or shift(centr2, img1, h)

In [115]:
def simple_hash(path = './dev_dataset'):
    list_of_images = os.listdir(path)
    k = len(list_of_images)
    hash_of_images = []
    
    for i in list_of_images:
        hash_of_images.append(creator(path+'/'+i))
        
    return hash_of_images

def compare_simple_hashes(hash_of_images):
    h = 10
    for i in range(k):
        for j in range(i+1,k):
            res[i][j] = is_equal_with_shift(hash_of_images[i], hash_of_images[j], h)
    return res

In [113]:
def test(path = './dev_dataset'):
    list_of_images = os.listdir(path)
    k = len(list_of_images)
    
    hash_of_images = []
    
    for i in list_of_images:
        hash_of_images.append(creator(path+'/'+i))
    
    for h in range(20):
        res = np.zeros((k,k))
        for i in range(k):
            for j in range(i+1,k):
                res[i][j] = is_equal_with_shift(hash_of_images[i], hash_of_images[j], h)

        matrix_of_mistakes = np.zeros((2,2))
        for i in range(k):
            for j in range(i+1,k):
                if list_of_images[i][:2] == list_of_images[j][:2]:
                    if res[i][j] == 1:
                        matrix_of_mistakes[0][0] += 1
                    else:
                        matrix_of_mistakes[1][0] += 1 
                elif res[i][j] == 1:
                    matrix_of_mistakes[0][1] += 1    
                else:
                    matrix_of_mistakes[1][1] += 1
                    
        prec, rec, F = f_mera(matrix_of_mistakes)
        print(f"{h}: F--{'%.2f'%F},prec--{'%.2f'%prec}, rec--{rec}, TP-{matrix_of_mistakes[0][0]}, FP-{matrix_of_mistakes[0][1]}, FN-{matrix_of_mistakes[1][0]}, TN-{matrix_of_mistakes[1][1]}, sum-{np.sum(res)}")

    return res

#### Определяю порог для расстояния Хэмминга
(с помощью f-score)

In [114]:
test5 = test('./global_test')

0: F--0.06,prec--1.00, rec--0.03125, TP-1.0, FP-0.0, FN-31.0, TN-1096.0, sum-1.0
1: F--0.12,prec--1.00, rec--0.0625, TP-2.0, FP-0.0, FN-30.0, TN-1096.0, sum-2.0
2: F--0.17,prec--0.75, rec--0.09375, TP-3.0, FP-1.0, FN-29.0, TN-1095.0, sum-4.0
3: F--0.26,prec--0.83, rec--0.15625, TP-5.0, FP-1.0, FN-27.0, TN-1095.0, sum-6.0
4: F--0.27,prec--0.50, rec--0.1875, TP-6.0, FP-6.0, FN-26.0, TN-1090.0, sum-12.0
5: F--0.30,prec--0.47, rec--0.21875, TP-7.0, FP-8.0, FN-25.0, TN-1088.0, sum-15.0
6: F--0.42,prec--0.52, rec--0.34375, TP-11.0, FP-10.0, FN-21.0, TN-1086.0, sum-21.0
7: F--0.43,prec--0.50, rec--0.375, TP-12.0, FP-12.0, FN-20.0, TN-1084.0, sum-24.0
8: F--0.51,prec--0.56, rec--0.46875, TP-15.0, FP-12.0, FN-17.0, TN-1084.0, sum-27.0
9: F--0.52,prec--0.52, rec--0.53125, TP-17.0, FP-16.0, FN-15.0, TN-1080.0, sum-33.0
10: F--0.56,prec--0.51, rec--0.625, TP-20.0, FP-19.0, FN-12.0, TN-1077.0, sum-39.0
11: F--0.51,prec--0.42, rec--0.65625, TP-21.0, FP-29.0, FN-11.0, TN-1067.0, sum-50.0
12: F--0.44,

Видим, что алгортим более менее нормально работает с порогом h = 10 

## Обьеденяем
phash - 21
just_hash - 11

In [157]:
path = './global_test'
list_of_images = os.listdir(path)
k = len(list_of_images)
simple_hash_images = simple_hash(path)
phash_of_images = pHash(path)

In [158]:
matrix_to_train = np.zeros((int(k*(k-1)/2), 3))

In [159]:
f = 0
for i in range(k):
    for j in range(i+1,k):
        matrix_to_train[f][0] = hamm(phash_of_images[i], phash_of_images[j])
        matrix_to_train[f][1] = is_equal_with_shift(simple_hash_images[i], simple_hash_images[j], 11)
        matrix_to_train[f][2] = list_of_images[i][:2] == list_of_images[j][:2]
        f += 1

In [166]:
df = pd.DataFrame(data=matrix_to_train, columns=['phash', 'simple_hash', 'target'])
df[df['phash'] < 21]

,phash,simple_hash,target
0,0.0,0.0,1.0
93,0.0,1.0,1.0
182,1.0,1.0,1.0
267,0.0,0.0,1.0
268,15.0,1.0,1.0
308,15.0,1.0,1.0
387,10.0,1.0,1.0
462,0.0,1.0,1.0
533,2.0,1.0,1.0
600,0.0,0.0,1.0


In [149]:
aa = df[df['phash'] > 21]
aa[aa['target'] == 1]

,phash,simple_hash,target
1023,29.0,0.0,1.0
1024,35.0,0.0,1.0
1100,32.0,1.0,1.0
1113,24.0,1.0,1.0
1122,31.0,1.0,1.0


In [137]:
bb = df[df['target'] == 0]
bb[bb['simple_hash']==1]

,phash,simple_hash,target
103,33.0,1.0,0.0
104,33.0,1.0,0.0
113,30.0,1.0,0.0
131,39.0,1.0,0.0
147,33.0,1.0,0.0
148,33.0,1.0,0.0
157,30.0,1.0,0.0
175,39.0,1.0,0.0
186,33.0,1.0,0.0
199,36.0,1.0,0.0


### Можем сделать вывод что поиск похожих изображений со смещением не может помочь pHash'у с детекцией

## Алгоритм SSIM
Может быть он нам помжет (хотя вряд ли ведь. ведь он смотрит на цвета изображения, а не на размещение)

In [73]:
def ssim(img1, img2):

    K = [0.01, 0.03]
    L = 65535          # 2 ** bpp(img1) - 1

    C1 = (K[0] * L) ** 2
    C2 = (K[1] * L) ** 2

    mu1 = np.mean(img1)
    mu2 = np.mean(img2)

    mu1_sq = np.multiply(mu1, mu1)
    mu2_sq = np.multiply(mu2, mu2)
    mu1_mu2 = np.multiply(mu1, mu2)

    sigma1_sq = np.cov(img1,img1) ** 2
    sigma2_sq = np.cov(img1,img1) ** 2
    sigma12 = np.cov(img1,img2) ** 2

    ssim_map = np.divide(np.multiply((2*mu1_mu2 + C1), (2*sigma12 + C2)), np.multiply((mu1_sq + mu2_sq + C1),(sigma1_sq + sigma2_sq + C2)))
    return np.mean(ssim_map)

def main(name1, name2):

    img1 = np.array(np.asarray(Image.open(f'{name1}').resize((1024,1024), Image.ANTIALIAS).convert(mode = 'L')),dtype=np.double)
    img2 = np.array(np.asarray(Image.open(f'{name2}').resize((1024,1024), Image.ANTIALIAS).convert(mode = 'L')),dtype=np.double)

    return ssim(img1, img2)

In [138]:
def test(path = './dev_dataset'):
    list_of_images = os.listdir(path)
    k = len(list_of_images)
    
    res = np.zeros((k,k))
    for i in range(k):
        for j in range(i+1,k):
            res[i][j] = abs(main(path+'/'+list_of_images[i], path+'/'+list_of_images[j]) - 1) 
        print('|',end='')
    print('LETS GOOOOO\n')
    
    for h in np.linspace(0.001, 0.1, 1000):
        matrix_of_mistakes = np.zeros((2,2))
        for i in range(k):
            for j in range(i+1,k):
                if list_of_images[i][:2] == list_of_images[j][:2]:
                    if res[i][j] < h:
                        matrix_of_mistakes[0][0] += 1
                    else:
                        matrix_of_mistakes[1][0] += 1 
                elif res[i][j] < h:
                    matrix_of_mistakes[0][1] += 1    
                else:
                    matrix_of_mistakes[1][1] += 1
                    
        prec, rec, F = f_mera(matrix_of_mistakes)
        print(f"{'%.4f'%h}: F--{'%.2f'%F},prec--{'%.2f'%prec}, rec--{'%.2f'%rec}, TP-{matrix_of_mistakes[0][0]}, FP-{matrix_of_mistakes[0][1]}, FN-{matrix_of_mistakes[1][0]}, TN-{matrix_of_mistakes[1][1]}")

    return res
              
%time test5 = test()

||||||||||||||||||LETS GOOOOO

0.0010: F--0.50,prec--1.00, rec--0.33, TP-2.0, FP-0.0, FN-4.0, TN-147.0
0.0011: F--0.50,prec--1.00, rec--0.33, TP-2.0, FP-0.0, FN-4.0, TN-147.0
0.0012: F--0.44,prec--0.67, rec--0.33, TP-2.0, FP-1.0, FN-4.0, TN-146.0
0.0013: F--0.44,prec--0.67, rec--0.33, TP-2.0, FP-1.0, FN-4.0, TN-146.0
0.0014: F--0.44,prec--0.67, rec--0.33, TP-2.0, FP-1.0, FN-4.0, TN-146.0
0.0015: F--0.44,prec--0.67, rec--0.33, TP-2.0, FP-1.0, FN-4.0, TN-146.0
0.0016: F--0.44,prec--0.67, rec--0.33, TP-2.0, FP-1.0, FN-4.0, TN-146.0
0.0017: F--0.40,prec--0.50, rec--0.33, TP-2.0, FP-2.0, FN-4.0, TN-145.0
0.0018: F--0.40,prec--0.50, rec--0.33, TP-2.0, FP-2.0, FN-4.0, TN-145.0
0.0019: F--0.40,prec--0.50, rec--0.33, TP-2.0, FP-2.0, FN-4.0, TN-145.0
0.0020: F--0.40,prec--0.50, rec--0.33, TP-2.0, FP-2.0, FN-4.0, TN-145.0
0.0021: F--0.40,prec--0.50, rec--0.33, TP-2.0, FP-2.0, FN-4.0, TN-145.0
0.0022: F--0.40,prec--0.50, rec--0.33, TP-2.0, FP-2.0, FN-4.0, TN-145.0
0.0023: F--0.40,prec--0.50, rec--

0.0236: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0237: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0238: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0239: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0240: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0241: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0242: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0243: F--0.43,prec--0.38, rec--0.50, TP-3.0, FP-5.0, FN-3.0, TN-142.0
0.0244: F--0.62,prec--0.50, rec--0.83, TP-5.0, FP-5.0, FN-1.0, TN-142.0
0.0245: F--0.62,prec--0.50, rec--0.83, TP-5.0, FP-5.0, FN-1.0, TN-142.0
0.0246: F--0.59,prec--0.45, rec--0.83, TP-5.0, FP-6.0, FN-1.0, TN-141.0
0.0247: F--0.59,prec--0.45, rec--0.83, TP-5.0, FP-6.0, FN-1.0, TN-141.0
0.0248: F--0.59,prec--0.45, rec--0.83, TP-5.0, FP-6.0, FN-1.0, TN-141.0
0.0249: F--0.59,prec--0.45, rec--0.83, TP-5.0, FP-6.0, FN-1.0, T

Аналогично находим, что хорошим порогом будет h = 0.0245

##### Теперь определим с чем конкретно не справляется phash

In [174]:
bad_index_pairs = []
f = 0
for i in range(k):
    for j in range(i+1,k):
        if hamm(phash_of_images[i], phash_of_images[j]) >= 21 and list_of_images[i][:2] == list_of_images[j][:2]:
            print(list_of_images[i],list_of_images[j])
            bad_index_pairs.append((i,j))



20-mod-2.jpg 20.JPG
20-mod-2.jpg 20_s.JPG
23.JPG 23_s.JPG
24.JPG 24_s.JPG
25.JPG 25_s.JPG


In [175]:
bad_index_pairs 

[(33, 34), (33, 35), (40, 41), (42, 43), (44, 45)]

In [176]:
ssim_hashes = np.zeros((len(bad_index_pairs),))
k=0
for i,j in bad_index_pairs:
    ssim_hashes[k] = abs(main(path+'/'+list_of_images[i], path+'/'+list_of_images[j]) - 1)
    print(list_of_images[i], list_of_images[j], ssim_hashes[k])
    k += 1

20-mod-2.jpg 20.JPG 0.07788731317142039
20-mod-2.jpg 20_s.JPG 0.07337145152720903
23.JPG 23_s.JPG 0.4928116594758991
24.JPG 24_s.JPG 0.24989504159027698
25.JPG 25_s.JPG 0.0273135858888216


Как видим значения намного выше заданого h(если повысим h, то f-score уменьшится). 
Поэтому было принято решение использовать только phash